# **Package & SQL Data Import**

In [1]:
# importing the necessary packages
import mysql.connector as mysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

# Connecting into the mysql Lahman db using the credentials from mysql workbench
db = mysql.connect(
    host="localhost",
    user="root",
    passwd="Mikejoe1",
    database="lahmansbaseballdb" 
    
)

print(db)

In [2]:
# Join Lahman tables together to build our dataset

# Calling the mysql.connector.cursor() to help execute our query statements
mycursor = db.cursor()

# Populating 'sql' with our query. We want to create a query that gives us all the necessary batting data
sql = """SELECT DISTINCT 
            batting.playerID, nameFirst, nameLast, batting.yearID, teams.teamID, name, debut,
            bats, birthCountry, birthState, batting.G, batting.AB, batting.R, batting.H, batting.2B, batting.3B, 
            batting.HR, batting.RBI, batting.SB, batting.CS, batting.BB, batting.SO, batting.IBB, batting.HBP, 
            batting.SH, batting.SF, batting.GIDP, G_p, G_c, G_1b, G_2b, G_3b, G_ss, G_lf, G_cf, G_rf, G_dh
            
            from batting
            
            JOIN people ON batting.playerID = people.playerID
            JOIN teams ON batting.team_ID = teams.ID
            JOIN appearances ON batting.playerID = appearances.playerID 
                and batting.yearID = appearances.yearID and batting.teamID = appearances.teamID"""


# Use .execute() to write a simple query to make sure everything is working
mycursor.execute(sql)

# Use .fetchall() to retrieve the result set of the query
myresult = mycursor.fetchall()

# **Data Cleansing & Transformation**

In [3]:
# Converting the query to a DataFrame so we can start setting the data up and add more calculations
df = pd.DataFrame(myresult)

# We need to add the proper column headers back into this DataFrame
df.columns = ['PlayerID', 'FirstName', 'LastName', 'Year', 'TeamID', 'Team', 'Debut', 'Bats', 'BirthCountry', 'BirthState',
                   'G', 'AB', 'R', 'H', '2B','3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 
                   'GIDP', 'G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf', 'G_dh']

df.head()

,PlayerID,FirstName,LastName,Year,TeamID,Team,Debut,Bats,BirthCountry,BirthState,...,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_dh
0,abercda01,Frank,Abercrombie,1871,TRO,Troy Haymakers,1871-10-21,None,USA,OK,...,0,0,0,0,0,1,0,0,0,0.0
1,addybo01,Bob,Addy,1871,RC1,Rockford Forest Citys,1871-05-06,L,CAN,ON,...,0,0,0,22,0,3,0,0,0,0.0
2,allisar01,Art,Allison,1871,CL1,Cleveland Forest Citys,1871-05-04,None,USA,PA,...,0,0,0,2,0,0,0,29,0,0.0
3,allisdo01,Doug,Allison,1871,WS3,Washington Olympics,1871-05-05,R,USA,PA,...,0,27,0,0,0,0,0,0,0,0.0
4,ansonca01,Cap,Anson,1871,RC1,Rockford Forest Citys,1871-05-06,R,USA,IA,...,0,5,1,2,20,0,1,0,0,0.0


In [4]:
### Cleaning up some columns
# Combining the FirstName & LastName columns to one
df['Name'] = df['FirstName'] + " " + df['LastName']

# Reordering the DataFrame columns to move the 'Name' column
cols = list(df)
cols.insert(1, cols.pop(cols.index('Name')))

# Repopulating df with new column order and rows
df = df.loc[:, cols]

# Converting the Debut to a datetime object so we can manipulate the date
df['Debut'] = pd.to_datetime(df['Debut'])

# Extracting the year from the debut column
df['DebutYear'] = pd.DatetimeIndex(df['Debut']).year

## Show the amount of nulls in each column of our DataFrame
# Creating an empty list for column names
names = []

# Creating an empty list for the number of null values in each column
values = []

# Checking for Missing Values
for col in df.columns:
    names.append(col)
    values.append(df[col].isnull().sum())
    print(names[-1],values[-1]) 

PlayerID 0
Name 38
FirstName 38
LastName 0
Year 0
TeamID 0
Team 0
Debut 8
Bats 1921
BirthCountry 75
BirthState 3025
G 0
AB 0
R 0
H 0
2B 0
3B 0
HR 0
RBI 756
SB 2368
CS 23540
BB 0
SO 2099
IBB 36649
HBP 2816
SH 6068
SF 36102
GIDP 25440
G_p 0
G_c 0
G_1b 0
G_2b 0
G_3b 0
G_ss 0
G_lf 0
G_cf 0
G_rf 0
G_dh 1267
DebutYear 8


In [5]:
### Replacing the nulls in the statistical columns with 0

# Dropping those players that do not have a complete name in the dataset
# This is caused by no string in the nameFirst column from the query
df = df[df['Name'].notna()]

## I want to keep the stats as modern as possible. So, I have chosen to start at 1955
## The most modern stat, IBB, was credited to hitters starting in 1955. So no nulls will appear.

# Creating a boolean column of whether the data is before or after 1955
df['Pre1955'] = df['Year']>=1955

# Removing the rows that contain 'False' from the df
df = df[df.Pre1955]

# Dropping the columns from the df we don't need anymore
df.drop(['FirstName', 'LastName', 'Pre1955', 'Debut'], axis=1, inplace=True)

df.head()

,PlayerID,Name,Year,TeamID,Team,Bats,BirthCountry,BirthState,G,AB,...,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_dh,DebutYear
37462,aaronha01,Hank Aaron,1955,ML1,Milwaukee Braves,R,USA,AL,153,602,...,0,0,27,0,0,30,0,104,0.0,1954.0
37463,aberal01,Al Aber,1955,DET,Detroit Tigers,L,USA,OH,39,17,...,0,0,0,0,0,0,0,0,0.0,1950.0
37464,abernte02,Ted Abernathy,1955,WS1,Washington Senators,R,USA,NC,40,26,...,0,0,0,0,0,0,0,0,0.0,1955.0
37465,abramca01,Cal Abrams,1955,BAL,Baltimore Orioles,L,USA,PA,118,309,...,0,3,0,0,0,13,58,46,0.0,1949.0
37466,adamsbo03,Bobby Adams,1955,CHA,Chicago White Sox,R,USA,CA,28,21,...,0,0,1,9,0,0,0,0,0.0,1946.0


In [6]:
## Show the amount of nulls in each column of our DataFrame
# Creating an empty list for column names
names = []

# Creating an empty list for the number of null values in each column
values = []

# Checking for Missing Values
for col in df.columns:
    names.append(col)
    values.append(df[col].isnull().sum())
    print(names[-1],values[-1])  

PlayerID 0
Name 0
Year 0
TeamID 0
Team 0
Bats 0
BirthCountry 0
BirthState 2504
G 0
AB 0
R 0
H 0
2B 0
3B 0
HR 0
RBI 0
SB 0
CS 0
BB 0
SO 0
IBB 0
HBP 0
SH 0
SF 0
GIDP 0
G_p 0
G_c 0
G_1b 0
G_2b 0
G_3b 0
G_ss 0
G_lf 0
G_cf 0
G_rf 0
G_dh 0
DebutYear 0


# **Stat Calculations**

In [8]:
# Calculating some popular statistics
# Batting Average
df['BA'] = df['H']/df['AB']

# On Base Percentage
df['OBP'] = (df['H'] + df['BB'] + df['HBP']) / (df['AB'] + df['BB'] + df['HBP'] + df['SF'])

# Singles
df['1B'] = df['H'] - (df['2B'] + df['3B'] + df['HR'])

# Total Bases
df['TB'] = df['1B'] + (2 * df['2B']) + (3 * df['3B']) + (4 * df['HR'])

# Slugging Percentage
df['SLG'] = df['TB'] / df['AB']

# On Base plus Slugginh
df['OPS'] = df['OBP'] + df['SLG']

# Plate Appearances
df['PA'] = df['AB'] + df['BB'] + df['HBP'] + df['SH'] + df['SF']

# Walk Percentage
df['BB%'] = df['BB'] /df['PA']

# Strikeout Percentage
df['K%'] = df['SO'] / df['PA']

# Isolated Power
df['ISO'] = df['SLG'] - df['BA']

# Batting Average on Balls in Play
df['BABIP'] = (df['H'] - df['HR']) / (df['AB'] - df['SO'] - df['HR'] + df['SF'])

# Runs Created
df['RC'] = ((df['H'] + df['BB']) * df['TB']) / (df['AB'] + df['BB'])

# Rounding the new calculations to desired decimal places
df = df.round({'BA':3, 'OBP':3, 'SLG':3, 'OPS':3, 'ISO':3, 'BABIP':3, 'RC':2, 'BB%':3, 'K%':3})

df = df[['PlayerID', 'Name', 'Year', 'TeamID', 'Team', 'DebutYear', 'Bats',
       'BirthCountry', 'BirthState', 'G', 'AB', 'PA', 'R', 'H', '1B', '2B', '3B', 'HR',
       'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'TB', 'IBB', 'HBP', 
        'SH', 'SF', 'GIDP', 'BB%', 'K%', 'ISO', 'BABIP', 'RC', 'G_p', 'G_c', 'G_1b', 
        'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf', 'G_dh']]

df.head()

,PlayerID,Name,Year,TeamID,Team,DebutYear,Bats,BirthCountry,BirthState,G,...,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_dh
37462,aaronha01,Hank Aaron,1955,ML1,Milwaukee Braves,1954.0,R,USA,AL,153,...,0,0,0,27,0,0,30,0,104,0.0
37463,aberal01,Al Aber,1955,DET,Detroit Tigers,1950.0,L,USA,OH,39,...,39,0,0,0,0,0,0,0,0,0.0
37464,abernte02,Ted Abernathy,1955,WS1,Washington Senators,1955.0,R,USA,NC,40,...,40,0,0,0,0,0,0,0,0,0.0
37465,abramca01,Cal Abrams,1955,BAL,Baltimore Orioles,1949.0,L,USA,PA,118,...,0,0,3,0,0,0,13,58,46,0.0
37466,adamsbo03,Bobby Adams,1955,CHA,Chicago White Sox,1946.0,R,USA,CA,28,...,0,0,0,1,9,0,0,0,0,0.0


In [9]:
# Converting some columns from floats to ints
df = df.astype({'RBI': 'int64', 'SB': 'int64', 'CS': 'int64', 'SO': 'int64', 'IBB': 'int64', 'HBP': 'int64',
            'SH': 'int64', 'SF': 'int64', 'GIDP': 'int64'})

print(df.dtypes)

PlayerID         object
Name             object
Year              int64
TeamID           object
Team             object
DebutYear       float64
Bats             object
BirthCountry     object
BirthState       object
G                 int64
AB                int64
PA              float64
R                 int64
H                 int64
1B                int64
2B                int64
3B                int64
HR                int64
RBI               int64
SB                int64
CS                int64
BB                int64
SO                int64
BA              float64
OBP             float64
SLG             float64
OPS             float64
TB                int64
IBB               int64
HBP               int64
SH                int64
SF                int64
GIDP              int64
BB%             float64
K%              float64
ISO             float64
BABIP           float64
RC              float64
G_p               int64
G_c               int64
G_1b              int64
G_2b            

In [10]:
# Writing the df to an excel file
df.to_excel("batting.xlsx")